# Homework 6: Nonstationary/Conditional Flood Frequency Analysis

### In this assignment, you will fit an LP3 distribution to annual maxima at Clark Fork below Missoula, MT, conditioned on climate indices using MOM and MLE. Code has been provided to download and process this data.

In [ ]:
!pip install dataretrieval
!pip install lmoments3

In [ ]:
from google.colab import drive
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
import dataretrieval.nwis as nwis
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

$\color{red}{\text{Edit the code below to download your own utils.py script with updated functions for nonstationary/conditional distribution fitting.}}$

In [ ]:
# allow access to google drive
drive.mount('/content/drive')

!cp "drive/MyDrive/Colab Notebooks/CE6280/Homeworks/HW6_utils.py" .
from HW6_utils import *

$\color{red}{\text{Run the code below with functions for downloading climate indices.}}$

This is an edited version of the code in [this repository](https://github.com/gabrielmpp/climate_indices/blob/master/climIndices/tools.py). I edited it because line 114 caused an error in Google Colab's Python/scipy version.

In [ ]:
from subprocess import call
import os
import requests
from pandas.errors import EmptyDataError
from datetime import datetime
from copy import deepcopy

SOURCES = ['NOAA', 'CPC']
PID = os.getpid()
TMP_FILE_PATH = os.environ['HOME'] + f'/temp_file_climIndices_{PID}.txt'


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


def exists(URL):
    r = requests.head(URL)
    return r.status_code == requests.codes.ok


def create_url(index, source):
    """
    Return the valid URL for download

    :param variable: string
    :param level: string
    :param date: datetime
    :return: sring
    """
    if source == 'NOAA':
        if 'nina' in index:
            base_url = 'https://psl.noaa.gov/data/correlation/{index}.anom.data'
        else:
            base_url = 'https://psl.noaa.gov/data/correlation/{index}.data'
    elif source == 'CPC':
        base_url = 'https://www.cpc.ncep.noaa.gov/data/indices/req{index}.for'
    else:
        raise ValueError("Source not supported")

    base_url = base_url.format(index=index)

    return base_url


def format_data(df, index, string_nan):
    colnames=['year']
    [colnames.append(i) for i in range(1,13)]
    df.columns = colnames
    df = df.set_index('year')
    df = df.unstack()
    df = df.reset_index()
    df.columns = ['month','year','value']
    df = df.sort_values(['year','month'])
    #df = df.replace('-99.99', np.NaN)
    #df = df.dropna()

    indexes = pd.date_range(start='{year:0d}-{month}-01'.format(year=int(df['year'].iloc[0]), month=int(df['month'].iloc[0])),
                            end='{year:0d}-{month}-31'.format(year=int(df['year'].iloc[-1]), month=int(df['month'].iloc[-1])),freq='M')
    df['time']=indexes
    df = df.set_index('time')
    df = df.drop(['month','year'], axis=1)
    df.columns = [index]
    df[index] = df[index].astype(float)
    df = df.replace(float(string_nan), np.nan)

    df = df.dropna()
    return df


def get_data(indices, source='NOAA'):
    def download_df(index, source):
        URL = create_url(index, source)
        if not exists(URL):
            print(URL)
            raise ValueError(f"URL does not exist for index {index}")

        call(["curl", "-s", "-o", TMP_FILE_PATH, URL], stdout=open(os.devnull, 'wb'))

    assert source in SOURCES, f'source {source} not valid.'
    _sources = deepcopy(SOURCES)
    df_list = []

    def format_datetime(x):
        return pd.Timestamp(datetime(day=1, month=x.month, year=x.year))

    for index in indices:
        for source in _sources:
            print(f'Trying source {source}')
            download_df(index, source)

            try:
                df_temp = pd.read_csv(TMP_FILE_PATH, sep='\s+', skiprows=[0], header=None)
            except (EmptyDataError, FileNotFoundError):
                print("Data is empty, trying another source")
            else:
                break
        try:
            df_temp
        except NameError:
            raise Exception(f'ClimIndices could not download index {index}')
        try:
            call(['rm', TMP_FILE_PATH])
        except:
            print('Could not remove temp file.')
        if source == 'CPC':
            string_nan = '999.9'
        else:
            df_nan = df_temp[df_temp.isnull()]
            string_nan = df_nan.iloc[0,0]
        df = df_temp.dropna()
        df = format_data(df, index, string_nan)
        df.index = [format_datetime(x) for x in df.index]
        df_list.append(df)

    df = pd.concat(df_list, axis=1)
    return df

$\color{red}{\text{Run the code below to download Ni$\tilde{\text{n}}$o 3.4 index time series and Pacific Decadal Oscillation time series.}}$

In [ ]:
df = get_data(['nina34','pdo'])

df.plot(subplots=True, sharex=True, title='Climate indices', legend='False')
plt.show()

$\color{red}{\text{Run the code below to clean the data, dropping nan values and no data values that have been stored as -99.99 or -9.9.}}$

In [ ]:
df

In [ ]:
nino34 = df['nina34'].iloc[np.where(df['nina34']!=-99.99)[0]]
nino34.plot()
nino34

In [ ]:
np.min(df['pdo'])

In [ ]:
pdo = df['pdo'].iloc[np.where(df['pdo']!=-9.9)[0]]
pdo.dropna(axis=0,inplace=True)
pdo.plot()
pdo

$\color{red}{\text{Run the code below to download all annual maxima at Clark Fork below Missoula MT over the period for which we have PDO and Ni$\tilde{\text{n}}$o 3.4 data in the months of January-April.}}$

In [ ]:
flow_df = nwis.get_record(sites='12353000', service='peaks', start='1950-01-01', end='2022-12-31') # Clark Fork below Missoula MT
flow_df

$\color{red}{\text{Run the code below to find the average PDO and Ni$\tilde{\text{n}}$o 3.4 indices from January-April each year and combine them into one data frame with the annual maxima.}}$

In [ ]:
df = pd.DataFrame({'pdo':pdo,'nino34':nino34})
df.dropna(axis=0,inplace=True)
df['Month'] = df.index.month
df['Year'] = df.index.year
np.where(df['Month']<=4)
df = df.iloc[np.where(df['Month']<=4)[0],:]
MeanIndices = df.groupby('Year').mean()
MeanIndices

In [ ]:
flow_df['Year'] = flow_df.index.year
flow_df = flow_df[['peak_va','Year']]
flow_df['nino34'] = MeanIndices['nino34'].values
flow_df['pdo'] = MeanIndices['pdo'].values
flow_df

## 1. Find the correlation between the annual maxima at Clark Fork below Missoula, MT and the winter season Ni$\tilde{\text{n}}$o 3.4 and PDO indices. Then plot all three variables in one plot with different colors for each time series. For visualization purposes, negate the climate indices if they are negatively correlated with the annual maxima. What do you observe?

## 2. Regress the log of the annual maxima on just the most correlated index and on both indices. Which model has a better Adjusted R$^2$? (Note: we're using the log of the annual maxima because we're going to fit a P3 distribution to the log of the data, so this regression will be useful for conditional MOM fitting).

## 3. Plot the time series of observed and fitted values from the preferred model (the one with a better Adjusted R$^2$). How does it look?

##4. Make a 4-panel figure of the preferred model's diagnostics (residuals vs. fitted, normal QQ plot, ACF and PACF). How do they look?

##5. Fit an LP3 distribution to the annual maxima using MOM **assuming stationarity** and estimate the 100-yr flood. Plot the time series with a line at the 100-yr flood estimate.

##6. Repeat question 5 but using MLE.

## 7. Fit an LP3 distribution to the annual maxima using MOM assuming the mean of the log of the data (and no other moments) of the distribution changes each year based on the value predicted by the preferred regression equation fit above. Plot the time series of annual maxima and 100-yr flood estimate each year. (Note: if the skew of the log of the data is negative, you should fit the distribution to -log(data), so the regression predictions of the mean should also be negated.)

##8. Repeat question 7 but using MLE, where the location parameter $\xi$ (and no other parameters) is a function of one or both climate indices (whichever combination worked better in the regression above). Plot the time series of annual maxima and 100-yr flood estimate each year.

## 9. Comment on when you think it would be best to use the stationary vs. conditional approach to FFA. What types of engineering decisions could each inform?